In [ ]:
!pip install pandas>=1.3 lmfit>=1.0.0 chemparse>=0.1.0 scikit-learn==1.3.2 compress_pickle>=2.1.0 torch lightning>=2.0.0 platformdirs


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
glasspy 0.5.3 requires scikit-learn>=1.5.0, but you have scikit-learn 1.3.2 which is incompatible.


In [ ]:
!pip install glasspy

  Using cached scikit_learn-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [ ]:
import pandas as pd
import lmfit
import chemparse
import compress_pickle
import sklearn
import torch
import lightning
import platformdirs
import glasspy

In [ ]:
#fmt: off
AtMol_translation = {
    "1": "H", "2": "He", "3": "Li", "4": "Be", "5": "B", "6": "C", "7": "N",
    "8": "O", "9": "F", "10": "Ne", "11": "Na", "12": "Mg", "13": "Al",
    "14": "Si", "15": "P", "16": "S", "17": "Cl", "18": "Ar", "19": "K",
    "20": "Ca", "21": "Sc", "22": "Ti", "23": "V", "24": "Cr", "25": "Mn",
    "26": "Fe", "27": "Co", "28": "Ni", "29": "Cu", "30": "Zn", "31": "Ga",
    "32": "Ge", "33": "As", "34": "Se", "35": "Br", "36": "Kr", "37": "Rb",
    "38": "Sr", "39": "Y", "40": "Zr", "41": "Nb", "42": "Mo", "43": "Tc",
    "44": "Ru", "45": "Rh", "46": "Pd", "47": "Ag", "48": "Cd", "49": "In",
    "50": "Sn", "51": "Sb", "52": "Te", "53": "I", "54": "Xe", "55": "Cs",
    "56": "Ba", "57": "La", "58": "Ce", "59": "Pr", "60": "Nd", "61": "Pm",
    "62": "Sm", "63": "Eu", "64": "Gd", "65": "Tb", "66": "Dy", "67": "Ho",
    "68": "Er", "69": "Tm", "70": "Yb", "71": "Lu", "72": "Hf", "73": "Ta",
    "74": "W", "75": "Re", "76": "Os", "77": "Ir", "78": "Pt", "79": "Au",
    "80": "Hg", "81": "Tl", "82": "Pb", "83": "Bi", "84": "Po", "85": "At",
    "86": "Rn", "87": "Fr", "88": "Ra", "89": "Ac", "90": "Th", "91": "Pa",
    "92": "U", "93": "Np", "94": "Pu",
}

SciGK_translation = {

    # Metadata
    "Analysis": {
        "info": "Indicates if the glass composition was obtained by"
                " chemical analysis",
        "rename": "ChemicalAnalysis",
        "convert": lambda x: True if x == "a" else False,
        "metadata": True,
    },
    "Author": {
        "info": "First author of the publication",
        "metadata": True,
    },
    "Year": {
        "info": "Year of the publication",
        "metadata": True,
    },

    # Viscosity
    "T1": {
        "info": "Temperature where viscosity is 1 Pa.s",
        "rename": "T0",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "T2": {
        "info": "Temperature where viscosity is 10 Pa.s",
        "rename": "T1",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "T3": {
        "info": "Temperature where viscosity is 100 Pa.s",
        "rename": "T2",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "T4": {
        "info": "Temperature where viscosity is 1000 Pa.s",
        "rename": "T3",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "T5": {
        "info": "Temperature where viscosity is 10000 Pa.s",
        "rename": "T4",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "T6": {
        "info": "Temperature where viscosity is 100000 Pa.s",
        "rename": "T5",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "T7": {
        "info": "Temperature where viscosity is 1000000 Pa.s",
        "rename": "T6",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "T8": {
        "info": "Temperature where viscosity is 10000000 Pa.s",
        "rename": "T7",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "T9": {
        "info": "Temperature where viscosity is 100000000 Pa.s",
        "rename": "T8",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "T10": {
        "info": "Temperature where viscosity is 1000000000 Pa.s",
        "rename": "T9",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "T11": {
        "info": "Temperature where viscosity is 10000000000 Pa.s",
        "rename": "T10",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "T12": {
        "info": "Temperature where viscosity is 100000000000 Pa.s",
        "rename": "T11",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "T13": {
        "info": "Temperature where viscosity is 1000000000000 Pa.s",
        "rename": "T12",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "V500": {
        "info": "Viscosity at 773 K",
        "rename": "Viscosity773K",
        "convert": lambda x: 10 ** (x - 1),
        "unit": "Pa.s",
    },
    "V600": {
        "info": "Viscosity at 873 K",
        "rename": "Viscosity873K",
        "convert": lambda x: 10 ** (x - 1),
        "unit": "Pa.s",
    },
    "V700": {
        "info": "Viscosity at 973 K",
        "rename": "Viscosity973K",
        "convert": lambda x: 10 ** (x - 1),
        "unit": "Pa.s",
    },
    "V800": {
        "info": "Viscosity at 1073 K",
        "rename": "Viscosity1073K",
        "convert": lambda x: 10 ** (x - 1),
        "unit": "Pa.s",
    },
    "V900": {
        "info": "Viscosity at 1173 K",
        "rename": "Viscosity1173K",
        "convert": lambda x: 10 ** (x - 1),
        "unit": "Pa.s",
    },
    "V1000": {
        "info": "Viscosity at 1273 K",
        "rename": "Viscosity1273K",
        "convert": lambda x: 10 ** (x - 1),
        "unit": "Pa.s",
    },
    "V1100": {
        "info": "Viscosity at 1373 K",
        "rename": "Viscosity1373K",
        "convert": lambda x: 10 ** (x - 1),
        "unit": "Pa.s",
    },
    "V1200": {
        "info": "Viscosity at 1473 K",
        "rename": "Viscosity1473K",
        "convert": lambda x: 10 ** (x - 1),
        "unit": "Pa.s",
    },
    "V1300": {
        "info": "Viscosity at 1573 K",
        "rename": "Viscosity1573K",
        "convert": lambda x: 10 ** (x - 1),
        "unit": "Pa.s",
    },
    "V1400": {
        "info": "Viscosity at 1673 K",
        "rename": "Viscosity1673K",
        "convert": lambda x: 10 ** (x - 1),
        "unit": "Pa.s",
    },
    "V1500": {
        "info": "Viscosity at 1773 K",
        "rename": "Viscosity1773K",
        "convert": lambda x: 10 ** (x - 1),
        "unit": "Pa.s",
    },
    "V1600": {
        "info": "Viscosity at 1873 K",
        "rename": "Viscosity1873K",
        "convert": lambda x: 10 ** (x - 1),
        "unit": "Pa.s",
    },
    "V1800": {
        "info": "Viscosity at 2073 K",
        "rename": "Viscosity2073K",
        "convert": lambda x: 10 ** (x - 1),
        "unit": "Pa.s",
    },
    "V2000": {
        "info": "Viscosity at 2273 K",
        "rename": "Viscosity2273K",
        "convert": lambda x: 10 ** (x - 1),
        "unit": "Pa.s",
    },
    "V2200": {
        "info": "Viscosity at 2473 K",
        "rename": "Viscosity2473K",
        "convert": lambda x: 10 ** (x - 1),
        "unit": "Pa.s",
    },
    # Temperatures
    "TG": {
        "info": "Glass transition temperature",
        "rename": "Tg",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "Tm": {
        "info": "Melting temperature",
        "rename": "Tmelt",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "TLiq": {
        "info": "Liquidus temperature",
        "rename": "Tliquidus",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "LPT": {
        "info": "Littletons softening temperature",
        "rename": "TLittletons",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "ANPT": {
        "info": "Annealing point",
        "rename": "TAnnealing",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "SPT": {
        "info": "Strain point",
        "rename": "Tstrain",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "Tsoft": {
        "info": "Softening point",
        "rename": "Tsoft",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "Mg": {
        "info": "Dilatometric softening temperature",
        "rename": "TdilatometricSoftening",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },

    # Optical
    "NUD300": {
        "info": "Abbe's number",
        "rename": "AbbeNum",
    },
    "ND300": {
        "info": "Refractive index",
        "rename": "RefractiveIndex",
    },
    "nd300low": {
        "info": "Refractive index measured at a wavelenght between 0.6 and 1 "
                "micron at 293 K",
        "rename": "RefractiveIndexLow",
    },
    "nd300hi": {
        "info": "Refractive index measured at a wavelenght greater than 1 "
                "micron at 293 K",
        "rename": "RefractiveIndexHigh",
    },
    "DNFC300": {
        "info": "Mean dispersion (nF - nC)",
        "rename": "MeanDispersion",
        "convert": lambda x: x * 1e-4,
    },

    # Electrical and dielectrical
    "EPS730": {
        "info": "Relative permittivity at ambient temperature anf frequency of"
                "1 MHz (or the nearest frequency in the range of 0.01 MHz to "
                "10 MHz)",
        "rename": "Permittivity",
    },
    "TGD730": {
        "info": "Tangent of loss angle",
        "rename": "TangentOfLossAngle",
        "convert": lambda x: x * 1e-4,
    },
    "TK100C": {
        "info": "Temperature where the specific electrical resistivity is 1"
                "MOhm.m",
        "rename": "TresistivityIs1MOhm.m",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "RO20": {
        "info": "Specific electrical resistivity measured at 273 K",
        "rename": "Resistivity273K",
        "convert": lambda x: 10 ** (x - 4),
        "unit": "Ohm.m",
    },
    "RO100": {
        "info": "Specific electrical resistivity measured at 373 K",
        "rename": "Resistivity373K",
        "convert": lambda x: 10 ** (x - 4),
        "unit": "Ohm.m",
    },
    "RO150": {
        "info": "Specific electrical resistivity measured at 423 K",
        "rename": "Resistivity423K",
        "convert": lambda x: 10 ** (x - 4),
        "unit": "Ohm.m",
    },
    "RO300": {
        "info": "Specific electrical resistivity measured at 573 K",
        "rename": "Resistivity573K",
        "convert": lambda x: 10 ** (x - 4),
        "unit": "Ohm.m",
    },
    "ro800": {
        "info": "Specific electrical resistivity measured at 1073 K",
        "rename": "Resistivity1073K",
        "convert": lambda x: 10 ** (x - 4),
        "unit": "Ohm.m",
    },
    "ro1000": {
        "info": "Specific electrical resistivity measured at 1273 K",
        "rename": "Resistivity1273K",
        "convert": lambda x: 10 ** (x - 4),
        "unit": "Ohm.m",
    },
    "ro1200": {
        "info": "Specific electrical resistivity measured at 1473 K",
        "rename": "Resistivity1473K",
        "convert": lambda x: 10 ** (x - 4),
        "unit": "Ohm.m",
    },
    "ro1400": {
        "info": "Specific electrical resistivity measured at 1673 K",
        "rename": "Resistivity1673K",
        "convert": lambda x: 10 ** (x - 4),
        "unit": "Ohm.m",
    },

    # Mechanical
    "MOD_UNG": {
        "info": "Young's Modulus",
        "rename": "YoungModulus",
        "unit": "GPa",
    },
    "MOD_SDV": {
        "info": "Shear Modulus",
        "rename": "ShearModulus",
        "unit": "GPa",
    },
    "MIKROTV": {
        "info": "Microhardness measured by Knoop or Vickers indentation",
        "rename": "Microhardness",
        "unit": "GPa",
    },
    "pois": {
        "info": "Poisson's ratio",
        "rename": "PoissonRatio",
    },

    # Density
    "DENSITY": {
        "info": "Density measured at 293 K",
        "rename": "Density293K",
        "unit": "g/cm3",
    },
    "dens800": {
        "info": "Density measured at 1073 K",
        "rename": "Density1073K",
        "unit": "g/cm3",
    },
    "dens1000": {
        "info": "Density measured at 1273 K",
        "rename": "Density1273K",
        "unit": "g/cm3",
    },
    "dens1200": {
        "info": "Density measured at 1473 K",
        "rename": "Density1473K",
        "unit": "g/cm3",
    },
    "dens1400": {
        "info": "Density measured at 1673 K",
        "rename": "Density1673K",
        "unit": "g/cm3",
    },

    # Thermal
    "cond220": {
        "info": "Thermal conductivity",
        "rename": "ThermalConductivity",
        "unit": "W/(m.K)",
    },
    "RTSH180": {
        "info": "Thermal shock resistance",
        "rename": "ThermalShockRes",
        "unit": "K",
    },
    "ANY_TEC": {
        "info": "Linear coefficient of thermal expansion measured below the "
                "glass transition temperature",
        "rename": "CTEbelowTg",
        "convert": lambda x: x * 1e-7,
        "unit": "1/K",
    },
    "TEC55": {
        "info": "Linear coefficient of thermal expansion measured at 328 +/- 10 K",
        "rename": "CTE328K",
        "convert": lambda x: x * 1e-7,
        "unit": "1/K",
    },
    "TEC100": {
        "info": "Linear coefficient of thermal expansion measured at 373 +/- 10 K",
        "rename": "CTE373K",
        "convert": lambda x: x * 1e-7,
        "unit": "1/K",
    },
    "TEC160": {
        "info": "Linear coefficient of thermal expansion measured at 433 +/- 10 K",
        "rename": "CTE433K",
        "convert": lambda x: x * 1e-7,
        "unit": "1/K",
    },
    "TEC210": {
        "info": "Linear coefficient of thermal expansion measured at 483 +/- 10 K",
        "rename": "CTE483K",
        "convert": lambda x: x * 1e-7,
        "unit": "1/K",
    },
    "TEC350": {
        "info": "Linear coefficient of thermal expansion measured at 623 +/- 10 K",
        "rename": "CTE623K",
        "convert": lambda x: x * 1e-7,
        "unit": "1/K",
    },
    "cp20": {
        "info": "Heat capacity at constant pressure measured at 293 K",
        "rename": "Cp293K",
        "unit": "J/(kg.K)",
    },
    "cp200": {
        "info": "Heat capacity at constant pressure measured at 473 K",
        "rename": "Cp473K",
        "unit": "J/(kg.K)",
    },
    "cp400": {
        "info": "Heat capacity at constant pressure measured at 673 K",
        "rename": "Cp673K",
        "unit": "J/(kg.K)",
    },
    "cp800": {
        "info": "Heat capacity at constant pressure measured at 1073 K",
        "rename": "Cp1073K",
        "unit": "J/(kg.K)",
    },
    "cp1000": {
        "info": "Heat capacity at constant pressure measured at 1273 K",
        "rename": "Cp1273K",
        "unit": "J/(kg.K)",
    },
    "cp1200": {
        "info": "Heat capacity at constant pressure measured at 1473 K",
        "rename": "Cp1473K",
        "unit": "J/(kg.K)",
    },
    "cp1400": {
        "info": "Heat capacity at constant pressure measured at 1673 K",
        "rename": "Cp1673K",
        "unit": "J/(kg.K)",
    },

    # Crystallization
    "Tn": {
        "info": "Nucleation temperature",
        "rename": "NucleationTemperature",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "Io": {
        "info": "Crystal nucleation rate",
        "rename": "NucleationRate",
        "convert": lambda x: x * 1e6,
        "unit": "1/(s.m3)",
    },
    "Tmax": {
        "info": "Temperature of maximum crystal growth velocity",
        "rename": "TMaxGrowthVelocity",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "Vmax": {
        "info": "Maximum crystal growth velocity",
        "rename": "MaxGrowthVelocity",
        "convert": lambda x: x / 100,
        "unit": "m/s",
    },
    "tcr": {
        "info": "DTA temperature of crystallization peak",
        "rename": "CrystallizationPeak",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },
    "tx": {
        "info": "DTA temperature of crystallization onset",
        "rename": "CrystallizationOnset",
        "convert": lambda x: x + 273.15,
        "unit": "K",
    },

    # Surface
    "any_sut": {
        "info": "Surface tension above the glass transition temperature",
        "rename": "SurfaceTensionAboveTg",
        "convert": lambda x: x / 1000,
        "unit": "J/m2",
    },
    "SUT900": {
        "info": "Surface tension at 1173 K",
        "rename": "SurfaceTension1173K",
        "convert": lambda x: x / 1000,
        "unit": "J/m2",
    },
    "SUT1200": {
        "info": "Surface tension at 1473 K",
        "rename": "SurfaceTension1473K",
        "convert": lambda x: x / 1000,
        "unit": "J/m2",
    },
    "SUT1300": {
        "info": "Surface tension at 1573 K",
        "rename": "SurfaceTension1573K",
        "convert": lambda x: x / 1000,
        "unit": "J/m2",
    },
    "SUT1400": {
        "info": "Surface tension at 1673 K",
        "rename": "SurfaceTension1673K",
        "convert": lambda x: x / 1000,
        "unit": "J/m2",
    },
}
#fmt: on

In [ ]:
"""This is the module to load available data in GlassPy.

Right now, the main source of GlassPy data is the SciGlass database. The
SciGlass database is available at https://github.com/epam/SciGlass licensed
under ODC Open Database License (ODbL). For a plain text version of this
database, see the for at https://github.com/drcassar/SciGlass. Data that ships
with GlassPy is the same as the data in the plain text fork.

Typical usage example:

  source = SciGlass()
  df = source.data

"""
import io
import shutil
import zipfile
from pathlib import Path
import requests
import pandas as pd
from platformdirs import user_data_dir

from glasspy.chemistry.convert import to_element_array, wt_to_mol


def _download_sciglass_data(path_dict):
    """Downloads the SciGlass database to your computer."""

    print("Downloading SciGlass database to your computer...")
    print("This is only required once and may take a few minutes.")

    record_id = "8287159"
    api_url = f"https://zenodo.org/api/records/{record_id}"
    response = requests.get(api_url, timeout=60)
    record_data = response.json()
    url = record_data["files"][0]["links"]["self"]

    download = requests.get(url, timeout=3600)
    zip_file = io.BytesIO(download.content)

    with zipfile.ZipFile(zip_file, "r") as zip_ref:
        for item in zip_ref.namelist():
            for name, path in path_dict.values():
                if name in item:
                    path.parent.mkdir(parents=True, exist_ok=True)
                    with zip_ref.open(item) as source_file:
                        with open(path, "wb") as target_file:
                            shutil.copyfileobj(source_file, target_file)

    print("Download completed!")


def _sciglass_path_dict():
    """Get the SciGlass file paths in your system."""

    data_dir = Path(user_data_dir("GlassPy"))

    path_dict = {
        "elements": ("AtMol", data_dir / "select_AtMol.csv.zip"),
        "properties": ("SciGK", data_dir / "select_SciGK.csv.zip"),
        "compounds": ("Gcomp", data_dir / "select_Gcomp.csv.zip"),
    }

    if not all(val[1].is_file() for val in path_dict.values()):
        _download_sciglass_data(path_dict)

    return path_dict


def sciglass_dbinfo():
    """Prints the SciGlass database information."""

    print()

    for key, value in SciGK_translation.items():
        name = value.get("rename", key)
        info = value.get("info", "")
        units = value.get("unit", None)
        units = f" ({units})" if units else ""
        meta = value.get("metadata", False)
        meta = " [metadata]" if meta else ""
        print(f"{name}: {info}{units}{meta}")


class SciGlass:
    """Loader of SciGlass data.

    Args:
      elements_cfg:
        Dictonary configuring how the `elements` information is collected. See
        docstring for `get_elements` method for more details.
      properties_cfg:
        Dictonary configuring how the `properties` information is collected. See
        docstring for `get_properties` method for more details.
      compounds_cfg:
        Dictonary configuring how the `compounds` information is collected. See
        docstring for `get_compounds` method for more details.
      autocleanup:
        If `True`, automatically delete columns of the final DataFrame that do
        not have any information (only zeros). Default value: True.
      metadata:
        If `True`, add the `metadata` information to the DataFrame. Default
        value: True.

    Attributes:
      data: DataFrame of the collected data.
    """
    def __init__(
        self,
        elements_cfg: dict | None = None,
        properties_cfg: dict | None = None,
        compounds_cfg: dict | None = None,
        autocleanup: bool = True,
        metadata: bool = True,
    ):
        path_dict = _sciglass_path_dict()

        if elements_cfg is None:
            elements_cfg = {
                "path": path_dict["elements"][1],
                "translate": AtMol_translation,
                "acceptable_sum_deviation": 1,
                "final_sum": 1,
            }

        if compounds_cfg is None:
            compounds_cfg = {
                "path": path_dict["compounds"][1],
                "acceptable_sum_deviation": 1,
                "final_sum": 1,
                "return_weight": False,
                "dropline": [
                    "",
                    "Al2O3+Fe2O3",
                    "MoO3+WO3",
                    "CaO+MgO",
                    "FeO+Fe2O3",
                    "Li2O+Na2O+K2O",
                    "Na2O+K2O",
                    "F2O-1",
                    "FemOn",
                    "HF+H2O",
                    "R2O",
                    "R2O3",
                    "R2O3",
                    "RO",
                    "RmOn",
                ],
            }

        if properties_cfg is None:
            properties_cfg = {
                "path": path_dict["properties"][1],
                "translate": SciGK_translation,
                "keep": self.available_properties(),
            }


        dfs = {}

        if properties_cfg:
            df = self.get_properties(**properties_cfg)
            dfs["property"] = df

        if elements_cfg:
            if "property" in dfs:
                df = self.get_elements(
                    IDs=dfs["property"].index, **elements_cfg
                )
            else:
                df = self.get_elements(**elements_cfg)
            dfs["elements"] = df

        if compounds_cfg:
            self.data = pd.concat(dfs, axis=1, join="inner")
            if len(self.data) > 0:
                df = self.get_compounds(IDs=self.data.index, **compounds_cfg)
            else:
                df = self.get_compounds(**compounds_cfg)
            dfs["compounds"] = df

        if metadata:
            metadata_cfg = {
                "path": path_dict["properties"][1],
                "translate": SciGK_translation,
                "keep": self.available_properties_metadata(),
            }

            df = self.get_properties(**metadata_cfg)
            dfs["metadata"] = df

            if "elements" in dfs:
                numelements = dfs["elements"].astype(bool).sum(axis=1)
                dfs["metadata"] = dfs["metadata"].assign(
                    NumberElements=numelements
                )

            if "compounds" in dfs:
                numcompounds = dfs["compounds"].astype(bool).sum(axis=1)
                dfs["metadata"] = dfs["metadata"].assign(
                    NumberCompounds=numcompounds
                )

            dfs["metadata"] = dfs["metadata"].convert_dtypes()

        # reordering
        dfs = {
            k: dfs[k]
            for k in ["elements", "compounds", "property", "metadata"]
            if k in dfs
        }

        self.data = pd.concat(dfs, axis=1, join="inner")

        if autocleanup:
            if "elements" in self.data.columns:
                self.remove_zero_sum_columns(scope="elements")

            if "compounds" in self.data.columns:
                self.remove_zero_sum_columns(scope="compounds")

    def get_properties(self, **kwargs):
        """Get properties information.

        Args:
          path : str
            String with the path to the database csv file.
          keep : list
            List of properties to keep in the final DataFrame.
          drop : list
            List of properties to remove from the final DataFrame.
          translate : dict
            Dictionary with the information on how to read and convert the
            properties. See variable `SciSK_translation` for an example.
          IDs : pd.Index
            IDs of the dataset to consider. Each glass in the SciGlass database
            has a glass number and a paper number. This ID used in GlassPy is an
            integer that merges both numbers
        """

        path_dict = _sciglass_path_dict()

        df = pd.read_csv(
            kwargs.get("path", path_dict["properties"][1]),
            sep="\t",
            low_memory=False,
        )
        df = df.assign(ID=lambda x: x.KOD * 100000000 + x.GLASNO)
        df = df.drop(["KOD", "GLASNO"], axis=1)
        translate = kwargs.get("translate", SciGK_translation)
        rename = {
            k: v["rename"] for k, v in translate.items() if "rename" in v
        }
        convert = {
            v.get("rename", k): v["convert"]
            for k, v in translate.items()
            if "convert" in v
        }
        df = self._process_df(df, rename=rename, convert=convert, **kwargs)
        return df

    def get_elements(self, **kwargs):
        """Get elemental atomic fraction information.

        Args:
          path : str
            String with the path to the database csv file.
          keep : list
            List of elements to keep in the final DataFrame.
          drop : list
            List of elements to remove from the final DataFrame.
          translate : dict
            Dictionary with the information on how to read and convert the
            elements. See variable `AtMol_translation` for an example.
          acceptable_sum_deviation : positive int or float
            The sum of all atomic fractions should be 100%.  However, due to
            float point errors or rounding errors, this sum will not be exactly
            100%. This argument controls the acceptable deviation of this sum in
            %. A value of 1 means that the sum of all atomic fractions can be
            between 99 and 101. All examples that are not within this range are
            discarted.
          final_sum : positive int or float
            The final sum of all atomic fractions is normalized to this value.
            Usual values are 1 if you want atomic fractions or 100 if you want
            atomic percentages.
          IDs : pd.Index
            IDs of the dataset to consider. Each glass in the SciGlass database
            has a glass number and a paper number. This ID used in GlassPy is an
            integer that merges both numbers
        """
        path_dict = _sciglass_path_dict()

        df = pd.read_csv(
            kwargs.get("path", path_dict["elements"][1]),
            sep="\t",
            low_memory=False,
        )
        df = df.assign(ID=lambda x: x.Kod * 100000000 + x.GlasNo)
        df = df.drop(["Kod", "GlasNo"], axis=1)
        translate = kwargs.get("translate", AtMol_translation)
        df = df.set_index("ID", drop=False)
        if "IDs" in kwargs:
            idx = df.index.intersection(kwargs["IDs"])
            df = df.loc[idx]
        df = self._process_df(df, rename=translate, **kwargs)
        return df

    def get_compounds(self, **kwargs):
        """Get compound information.

        Args:
          path : str
            String with the path to the database csv file.
          keep : list
            List of compounds to keep in the final DataFrame.
          drop : list
            List of compounds to remove from the final DataFrame.
          acceptable_sum_deviation : positive int or float
            The sum of all compound fractions should be 100%.  However, due to
            float point errors or rounding errors, this sum will not be exactly
            100%. This argument controls the acceptable deviation of this sum in
            %. A value of 1 means that the sum of all compound fractions can be
            between 99 and 101. All examples that are not within this range are
            discarted.
          final_sum : positive int or float
            The final sum of all compound fractions is normalized to this value.
            Usual values are 1 if you want compound fractions or 100 if you want
            compound percentages.
          return_weight : bool
            If `True`, the chemical information stored in the DataFrame will be
            in weight%. Otherwise it will be in mol%.
          IDs : pd.Index
            IDs of the dataset to consider. Each glass in the SciGlass database
            has a glass number and a paper number. This ID used in GlassPy is an
            integer that merges both numbers
        """
        path_dict = _sciglass_path_dict()

        df = pd.read_csv(
            kwargs.get("path", path_dict["compounds"][1]),
            sep="\t",
            low_memory=False,
        )
        df = df.assign(ID=lambda x: x.Kod * 100000000 + x.GlasNo)
        df = df.drop(["Kod", "GlasNo"], axis=1)
        df = df.set_index("ID", drop=True)

        if "IDs" in kwargs:
            idx = df.index.intersection(kwargs["IDs"])
            df = df.loc[idx]

        df = df["Composition"].str.slice(start=1, stop=-1)
        df = df.str.split("\x7f")

        add = 0 if kwargs.get("return_weight", False) else 1

        compound_list = []
        for row in df.values:
            compound_list.append(
                {
                    row[n * 4]: float(row[(n * 4 + 2 + add)])
                    for n in range(len(row) // 4)
                }
            )

        data = (
            pd.DataFrame(compound_list)
            .dropna(axis=1, how="all")
            .assign(ID=df.index)
            .dropna(axis=0, how="all")
            .fillna(0)
        )
        df = self._process_df(data, **kwargs)
        return df

    @staticmethod
    def _process_df(df, **kwargs):
        """Function to process the DataFrame."""

        df = df.drop_duplicates("ID", keep=False)
        df = df.set_index("ID", drop=True)

        if "rename" in kwargs:
            df = df.rename(columns=kwargs["rename"])
        if "keep" in kwargs:
            df = df.reindex(kwargs["keep"], axis=1)
        if "must_have_or" in kwargs:
            strings = [f"{el} > 0" for el in kwargs["must_have_or"]]
            query = " or ".join(strings)
            df = df.query(query)
        if "must_have_and" in kwargs:
            strings = [f"{el} > 0" for el in kwargs["must_have_and"]]
            query = " and ".join(strings)
            df = df.query(query)
        if "drop" in kwargs:
            df = df.drop(kwargs["drop"], axis=1, errors="ignore")
        if "drop_compound_with_element" in kwargs:
            drop_cols = []
            for col in df.columns:
                for el in kwargs["drop_compound_with_element"]:
                    if el in col:
                        df = df.loc[df[col] == 0]
                        drop_cols.append(col)
                        break
            df = df.drop(drop_cols, axis=1, errors="ignore")
        if "dropline" in kwargs:
            for col in kwargs["dropline"]:
                if col in df.columns:
                    df = df.loc[df[col] == 0]
            df = df.drop(kwargs["dropline"], axis=1, errors="ignore")
        if "convert" in kwargs:
            for k, v in kwargs["convert"].items():
                if k in df.columns:
                    df[k] = df[k].apply(v)
        if "acceptable_sum_deviation" in kwargs:
            diff = df.sum(axis=1) - 100
            thres = kwargs["acceptable_sum_deviation"]
            df = df.loc[diff.between(-thres, thres)]
        if "final_sum" in kwargs:
            df = df.divide(df.sum(axis=1), axis=0) * kwargs["final_sum"]

        df = df.dropna(axis=0, how="all")
        df = df.dropna(axis=1, how="all")

        return df

    def remove_zero_sum_columns(self, scope="compounds"):
        """Removes all columns that have zero sum from the `data` attribute."""

        zero_sum_cols = self.data[scope].sum(axis=0) == 0
        drop_cols = self.data[scope].columns[zero_sum_cols].tolist()
        drop_cols = [(scope, el) for el in drop_cols]
        self.data.drop(drop_cols, axis=1, inplace=True)

    def remove_duplicate_composition(
        self, scope="elements", decimals=3, aggregator="median"
    ):
        """Remove duplicate compositions from the `data` attribute.

        Note that the original ID and the metadata are lost upon this operation.
        """

        assert scope in ["elements", "compounds"]
        assert "property" in self.data.columns.levels[0]
        assert scope in self.data.columns.levels[0]

        comp_cols = self.data[scope].columns.to_list()
        prop_cols = self.data["property"].columns.to_list()
        df = self.data[[scope, "property"]].droplevel(0, axis=1)
        df[comp_cols] = df[comp_cols].round(decimals)
        grouped = df.groupby(comp_cols, sort=False)
        df = getattr(grouped, aggregator)().reset_index()
        df = {scope: df[comp_cols], "property": df[prop_cols]}
        self.data = pd.concat(df, axis=1, join="inner")

    def elements_from_compounds(self, final_sum=1, compounds_in_weight=False):
        """Create atomic fraction information from compound information.

        Args:
          final_sum : positive int or float
            The final sum of all atomic fractions is normalized to this value.
            Usual values are 1 if you want atomic fractions or 100 if you want
            atomic percentages. Default value is 1.
          compounds_in_weight : bool
            If `True`, then assume that the compounds fractions are in weight%,
            otherwise assume that the compounds fractions are in mol%. Default
            value is `False`.
        """

        assert "compounds" in self.data.columns.levels[0]
        assert "elements" not in self.data.columns.levels[0]

        chemarray = to_element_array(
            self.data["compounds"], rescale_to_sum=final_sum
        )

        if compounds_in_weight:
            chemarray = wt_to_mol(chemarray, chemarray.cols, final_sum)

        el_df = pd.DataFrame(
            chemarray,
            columns=chemarray.cols,
            index=self.data["compounds"].index,
        )
        dfs = {k: self.data[k] for k in self.data.columns.levels[0]}
        dfs["elements"] = el_df

        # reordering
        dfs = {
            k: dfs[k]
            for k in ["elements", "compounds", "property", "metadata"]
            if k in dfs.keys()
        }

        self.data = pd.concat(dfs, axis=1, join="inner")

    @staticmethod
    def available_properties():
        """Returns a list of available properties."""

        metadata = [
            v.get("rename", k)
            for k, v in SciGK_translation.items()
            if v.get("metadata", False)
        ]

        return [
            SciGK_translation[k].get("rename", k)
            for k in SciGK_translation
            if SciGK_translation[k].get("rename", k) not in metadata
        ]

    @staticmethod
    def available_properties_metadata():
        """Returns a list of available properties metadata."""

        return [
            v.get("rename", k)
            for k, v in SciGK_translation.items()
            if v.get("metadata", False)
        ]

    @staticmethod
    def database_info():
        """Prints database information."""
        sciglass_dbinfo()

In [ ]:
__all__ = ["SciGlass", "sciglass_dbinfo"]
source = SciGlass()
df = source.data
df.head()

This is only required once and may take a few minutes.
Download completed!


elements                                                         \
                   H   Li   Be    B    C    N         O    F        Na   Mg   
ID                                                                            
20400020000      0.0  0.0  0.0  0.0  0.0  0.0  0.666667  0.0  0.000000  0.0   
20500020001      0.0  0.0  0.0  0.0  0.0  0.0  0.579213  0.0  0.196815  0.0   
20500020002      0.0  0.0  0.0  0.0  0.0  0.0  0.580869  0.0  0.193449  0.0   
20500020003      0.0  0.0  0.0  0.0  0.0  0.0  0.581986  0.0  0.187167  0.0   
20500020004      0.0  0.0  0.0  0.0  0.0  0.0  0.583672  0.0  0.183080  0.0   

             ...              property                      \
             ... SurfaceTensionAboveTg SurfaceTension1173K   
ID           ...                                             
20400020000  ...                   NaN                 NaN   
20500020001  ...                   NaN                 NaN   
20500020002  ...                   NaN                 NaN   
20500020003  ...                   NaN                 NaN   
20500020004  ...                   NaN                 NaN   

                                                                         \
            SurfaceTension1473K SurfaceTension1573K SurfaceTension1673K   
ID                                                                        
20400020000                 NaN                 NaN                 NaN   
20500020001                 NaN                 NaN                 NaN   
20500020002                 NaN                 NaN                 NaN   
20500020003                 NaN                 NaN                 NaN   
20500020004                 NaN                 NaN                 NaN   

                    metadata                                        \
            ChemicalAnalysis           Author  Year NumberElements   
ID                                                                   
20400020000            False  Volarovich M.P.  1936              2   
20500020001            False         Hoj J.W.  1992              5   
20500020002            False         Hoj J.W.  1992              5   
20500020003            False         Hoj J.W.  1992              5   
20500020004            False         Hoj J.W.  1992              5   

                             
            NumberCompounds  
ID                           
20400020000               1  
20500020001               4  
20500020002               4  
20500020003               4  
20500020004               4  

[5 rows x 793 columns]

In [ ]:
df_filtered = df.loc[:, df.columns.get_level_values(0).isin(['property', 'elements'])]
df_filtered.columns

MultiIndex([('elements',                     'H'),
            ('elements',                    'Li'),
            ('elements',                    'Be'),
            ('elements',                     'B'),
            ('elements',                     'C'),
            ('elements',                     'N'),
            ('elements',                     'O'),
            ('elements',                     'F'),
            ('elements',                    'Na'),
            ('elements',                    'Mg'),
            ...
            ('property',        'NucleationRate'),
            ('property',    'TMaxGrowthVelocity'),
            ('property',     'MaxGrowthVelocity'),
            ('property',   'CrystallizationPeak'),
            ('property',  'CrystallizationOnset'),
            ('property', 'SurfaceTensionAboveTg'),
            ('property',   'SurfaceTension1173K'),
            ('property',   'SurfaceTension1473K'),
            ('property',   'SurfaceTension1573K'),
            ('p

In [ ]:


'''
# Supondo que df seja o DataFrame com MultiIndex
# Definir os nomes das propriedades que você quer manter
# glass transition temperature -> Tg
# Young's Modulus -> YoungModulus
# coefficient of thermal expansion -> CTEbelowTg/CTE328K/CTE373K/CTE433K/CTE483K/CTE623K
# refractive index -> RefractiveIndex
"info": "Liquidus temperature",
        "rename": "Tliquidus",

"NUD300": {
   "info": "Abbe's number",
   "rename": "AbbeNum",
'''

desired_properties = ['Tg', 'Tliquidus', 'YoungModulus', 'CTEbelowTg', 'CTE328K',
                      'CTE373K', 'CTE433K', 'CTE483K', 'CTE623K', 'RefractiveIndex', 'AbbeNum']

df_filtered = df.loc[:, (df.columns.get_level_values(0) == 'elements') |
                        ((df.columns.get_level_values(0) == 'property') &
                         df.columns.get_level_values(1).isin(desired_properties))]
print(df_filtered)

              elements                                                         \
                     H   Li   Be         B    C    N         O    F        Na   
ID                                                                              
20400020000        0.0  0.0  0.0  0.000000  0.0  0.0  0.666667  0.0  0.000000   
20500020001        0.0  0.0  0.0  0.000000  0.0  0.0  0.579213  0.0  0.196815   
20500020002        0.0  0.0  0.0  0.000000  0.0  0.0  0.580869  0.0  0.193449   
20500020003        0.0  0.0  0.0  0.000000  0.0  0.0  0.581986  0.0  0.187167   
20500020004        0.0  0.0  0.0  0.000000  0.0  0.0  0.583672  0.0  0.183080   
...                ...  ...  ...       ...  ...  ...       ...  ...       ...   
4493300611694      0.0  0.0  0.0  0.000000  0.0  0.0  0.625485  0.0  0.000000   
4493300611695      0.0  0.0  0.0  0.001948  0.0  0.0  0.637540  0.0  0.000000   
4493300611696      0.0  0.0  0.0  0.000000  0.0  0.0  0.635921  0.0  0.000000   
4493300611697      0.0  0.0 

In [ ]:
df_filtered.columns = df_filtered.columns.droplevel(0)

# Exibir as colunas filtradas
print(df_filtered.columns)

Index(['H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P',
       'S', 'Cl', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni',
       'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Rb', 'Sr', 'Y', 'Zr', 'Nb',
       'Mo', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Cs',
       'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er',
       'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Pt', 'Au', 'Hg', 'Tl', 'Pb',
       'Bi', 'Th', 'U', 'Tg', 'Tliquidus', 'AbbeNum', 'RefractiveIndex',
       'YoungModulus', 'CTEbelowTg', 'CTE328K', 'CTE373K', 'CTE433K',
       'CTE483K', 'CTE623K'],
      dtype='object')


##Filtro para oxidos

In [ ]:
import numpy as np
import pandas as pd

def oxide_filter(data, save_path):
    # Elementos proibidos
    forbidden_elements = ['S', 'H', 'C', 'Pt', 'Au', 'F', 'Cl', 'N', 'Br', 'I']

    # Filtra para manter apenas as linhas onde oxigênio (O) >= 0.3
    df_filtered = data[data['O'] >= 0.3]

    # Filtra para remover as linhas onde algum dos elementos proibidos tem quantidade não nula
    df_filtered = df_filtered[(df_filtered[forbidden_elements] == 0).all(axis=1)]

    # Aplica a verificação nas colunas de CTE para substituir valores negativos por NaN
    df_filtered = replace_negative_CTEs(df_filtered)

    # Aplica a transformação logarítmica nas colunas de CTE
    df_filtered = log_transform_CTEs(df_filtered)

    # Chama a função para filtrar as propriedades
    df = filter_Tg_E_CTE_Nd(df_filtered)

    # Salva o DataFrame filtrado em um arquivo CSV
    file_name = save_path + "/Oxides_prop.csv"
    df.to_csv(file_name, header=True, index=False)

    return df

def replace_negative_CTEs(data):
    """
    Substitui valores menores que zero nas colunas de CTE por NaN.
    """
    columns_to_check = ['CTEbelowTg', 'CTE328K', 'CTE373K', 'CTE433K', 'CTE483K', 'CTE623K']

    # Verifica e substitui os valores menores que zero por NaN em todas as colunas especificadas
    for column in columns_to_check:
        data.loc[data[column] < 0, column] = np.nan
    return data

def log_transform_CTEs(data):
    """
    Aplica a transformação logarítmica nas colunas de CTE.
    As colunas devem ter valores positivos e não nulos.
    """
    columns_to_transform = ['CTEbelowTg', 'CTE328K', 'CTE373K', 'CTE433K', 'CTE483K', 'CTE623K']

    # Substitui valores negativos ou zero por NaN e aplica o log10
    for column in columns_to_transform:
        data.loc[data[column] <= 0, column] = np.nan
        data[column] = np.log10(data[column])

    return data

def filter_Tg_E_CTE_Nd(data):
    # Propriedades que devem ser verificadas
    properties = ['Tg', 'Tliquidus', 'YoungModulus', 'CTEbelowTg', 'CTE328K',
                  'CTE373K', 'CTE433K', 'CTE483K', 'CTE623K', 'RefractiveIndex', 'AbbeNum']

    # Filtra para manter apenas as linhas onde pelo menos uma das propriedades não é nula
    df_filtered = data[(data[properties] != 0).any(axis=1)]

    return df_filtered


In [ ]:
import os

save_path = "/content/"

def main():
    data = df_filtered

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    oxide_filter(data, save_path)


if __name__ == "__main__":
    main()

In [ ]:
oxide = pd.read_csv("/content/Oxides_prop.csv")
oxide

,H,Li,Be,B,C,N,O,F,Na,Mg,...,Tliquidus,AbbeNum,RefractiveIndex,YoungModulus,CTEbelowTg,CTE328K,CTE373K,CTE433K,CTE483K,CTE623K
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.666667,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.579213,0.0,0.196815,0.000000,...,NaN,NaN,NaN,NaN,-4.826814,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.580869,0.0,0.193449,0.000000,...,NaN,NaN,NaN,NaN,-4.913640,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.581986,0.0,0.187167,0.000000,...,NaN,NaN,NaN,NaN,-4.889410,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.583672,0.0,0.183080,0.000000,...,NaN,NaN,NaN,NaN,-4.826814,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205892,0.0,0.0,0.0,0.000000,0.0,0.0,0.625485,0.0,0.000000,0.049125,...,NaN,NaN,NaN,99.500000,-5.327902,NaN,NaN,NaN,-5.327902,NaN
205893,0.0,0.0,0.0,0.001948,0.0,0.0,0.637540,0.0,0.000000,0.009932,...,1498.15,NaN,NaN,81.000000,-5.405607,NaN,NaN,NaN,-5.405607,NaN
205894,0.0,0.0,0.0,0.000000,0.0,0.0,0.635921,0.0,0.000000,0.000000,...,1486.15,NaN,NaN,80.000000,-5.340084,NaN,NaN,NaN,-5.340084,NaN
205895,0.0,0.0,0.0,0.014544,0.0,0.0,0.622226,0.0,0.035890,0.000000,...,NaN,NaN,NaN,81.199997,-5.394695,NaN,NaN,NaN,-5.394695,NaN


In [ ]:
# Definir as colunas que você deseja contar os valores não nulos
columns_to_count = ['Tg', 'Tliquidus', 'YoungModulus', 'CTEbelowTg', 'CTE328K',
                  'CTE373K', 'CTE433K', 'CTE483K', 'CTE623K', 'RefractiveIndex', 'AbbeNum']

# Contar os valores não nulos em cada uma dessas colunas
non_null_counts = oxide[columns_to_count].count()

# Exibir o resultado
print(non_null_counts)

Tg                 59987
Tliquidus          36498
YoungModulus       14527
CTEbelowTg         56937
CTE328K             2122
CTE373K             1911
CTE433K            17067
CTE483K            14431
CTE623K             1098
RefractiveIndex    50793
AbbeNum            24784
dtype: int64


In [ ]:
def remove_extremes(df, columns_to_filter):
    """
    Remove linhas com valores extremos (abaixo do 0.05% e acima do 99.95%)
    para as colunas especificadas, mantendo linhas com valores NaN.
    """
    # Calcula os limites inferior e superior para cada coluna
    lower_bound = df[columns_to_filter].quantile(0.0005)
    upper_bound = df[columns_to_filter].quantile(0.9995)

    print(lower_bound)
    print(upper_bound)

    # Filtra o DataFrame para manter apenas os valores dentro dos limites, preservando NaNs
    filtered_df = df.copy()  # Cria uma cópia para não modificar o original
    for column in columns_to_filter:
        filtered_df = filtered_df[
            (filtered_df[column].isna()) |  # Mantém os valores NaN
            ((filtered_df[column] >= lower_bound[column]) &
             (filtered_df[column] <= upper_bound[column]))
        ]

    # Redefine o índice
    filtered_df = filtered_df.reset_index(drop=True)

    return filtered_df

# Especificando as colunas que você deseja filtrar
columns_to_filter = ['Tg', 'Tliquidus', 'YoungModulus', 'CTEbelowTg', 'CTE328K',
                  'CTE373K', 'CTE433K', 'CTE483K', 'CTE623K', 'RefractiveIndex', 'AbbeNum']

# Chamando a função
filtered_df = remove_extremes(oxide, columns_to_filter)

# Exibindo o DataFrame filtrado
print(filtered_df)

Tg                 378.143000
Tliquidus          595.150000
YoungModulus         3.632600
CTEbelowTg          -6.886057
CTE328K             -7.186656
CTE373K             -7.783854
CTE433K             -7.258826
CTE483K             -6.765470
CTE623K             -7.305648
RefractiveIndex      1.400000
AbbeNum             10.443065
Name: 0.0005, dtype: float64
Tg                 1257.178000
Tliquidus          3062.653000
YoungModulus        164.520025
CTEbelowTg           -4.347213
CTE328K              -4.423869
CTE373K              -4.428291
CTE433K              -4.112504
CTE483K              -4.597249
CTE623K              -3.856712
RefractiveIndex       2.762080
AbbeNum              71.960851
Name: 0.9995, dtype: float64
          H   Li   Be         B    C    N         O    F        Na        Mg  \
0       0.0  0.0  0.0  0.000000  0.0  0.0  0.666667  0.0  0.000000  0.000000   
1       0.0  0.0  0.0  0.000000  0.0  0.0  0.579213  0.0  0.196815  0.000000   
2       0.0  0.0  0.0  0.000000

In [ ]:
import os

save_path = "/content/"

def main():
    data = filtered_df

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    file_name = save_path+"/oxide_clean.csv"

    data.to_csv(file_name, header=True, index=False)


if __name__ == "__main__":
    main()